In [ ]:
#import libraries
import pandas as pd
import os
import sys
import time
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

#display settings
pd.set_option('display.max_columns', None)

In [ ]:
#ini Dask client
cluster = LocalCluster(n_workers=10)
client = Client(cluster)